In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import r2_score

In [ ]:
import os
# find all the txt files in the dataset folder
inputs = []
for file in os.listdir("train"):
    if file.endswith(".txt"):
        inputs.append(os.path.join("train", file))
        
# concatanate all txt files in a file called merged_file.txt
with open('merged_file1.txt', 'w') as outfile:
    for fname in inputs:
        with open(fname, encoding="utf-8", errors='ignore') as infile:
            for line in infile:
                outfile.write(line)
            
            outfile.write("\n")
            

In [167]:
pd.set_option('display.max_columns',None)

In [168]:
def Data_Transforming(path: str = "/path/to/text/"):
    """
    This function takes a path string to a txt file and transforms data using Regular Expression and loads it into
    a Pandas DataFrame.

    :param      path (optional): str, relative path of the CSV file

    :return     df: pd.DataFrame
    """
    
    data=pd.read_csv(f"{path}",names=['a','b'])
    data['ID']=data['a'].str.extract(r"^ID\s+(.*)").dropna().reset_index(drop=True)
    data['Delivery_person_ID']=data['a'].str.extract(r"^Delivery_person_ID\s+(.*)").dropna().reset_index(drop=True)
    data['Delivery_person_Age']=data['a'].str.extract(r"^Delivery_person_Age\s+(.*)").dropna().reset_index(drop=True)
    data['Delivery_person_Ratings']=data['a'].str.extract(r"^Delivery_person_Ratings\s+(.*)").dropna().reset_index(drop=True)
    data['Restaurant_latitude']=data['a'].str.extract(r"^Restaurant_latitude\s+(.*)").dropna().reset_index(drop=True)
    data['Restaurant_longitude']=data['a'].str.extract(r"^Restaurant_longitude\s+(.*)").dropna().reset_index(drop=True)
    data['Delivery_location_latitude']=data['a'].str.extract(r"^Delivery_location_latitude\s+(.*)").dropna().reset_index(drop=True)
    data['Delivery_location_longitude']=data['a'].str.extract(r"^Delivery_location_longitude\s+(.*)").dropna().reset_index(drop=True)
    data['Order_Date']=data['a'].str.extract(r"^Order_Date\s+(.*)").dropna().reset_index(drop=True)
    data['Time_Orderd']=data['a'].str.extract(r"^Time_Orderd\s+(.*)").dropna().reset_index(drop=True)
    data['Time_Order_picked']=data['a'].str.extract(r"^Time_Order_picked\s+(.*)").dropna().reset_index(drop=True)
    data['Weather conditions']=data['a'].str.extract(r"^Weather conditions\s+(.*)").dropna().reset_index(drop=True)
    data['Road_traffic_density']=data['a'].str.extract(r"^Road_traffic_density\s+(.*)").dropna().reset_index(drop=True)
    data['Vehicle_condition']=data['a'].str.extract(r"^Vehicle_condition\s+(.*)").dropna().reset_index(drop=True)
    data['Type_of_order']=data['a'].str.extract(r"^Type_of_order\s+(.*)").dropna().reset_index(drop=True)
    data['Type_of_vehicle']=data['a'].str.extract(r"^Type_of_vehicle\s+(.*)").dropna().reset_index(drop=True)
    data['multiple_deliveries']=data['a'].str.extract(r"^multiple_deliveries\s+(.*)").dropna().reset_index(drop=True)
    data['Festival']=data['a'].str.extract(r"^Festival\s+(.*)").dropna().reset_index(drop=True)
    data['City']=data['a'].str.extract(r"^City\s+(.*)").dropna().reset_index(drop=True)
    data['Time_taken']=data['a'].str.extract(r"^Time_taken\s\(.+\)\s+(.*)").dropna().reset_index(drop=True)
    data.drop(labels=['a','b'],axis=1,inplace=True)
    data.dropna(inplace=True)
    return data

In [169]:
train_df = Data_Transforming('merged_file1.txt')
train_df

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken
0,0x4607,INDORES13DEL02,37.000000,4.900000,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30,11:45,Sunny,High,2,Snack,motorcycle,0.000000,No,Urban,24.000000
1,0xb379,BANGRES18DEL02,34.000000,4.500000,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45,19:50,Stormy,Jam,2,Snack,scooter,1.000000,No,Metropolitian,33.000000
2,0x5d6d,BANGRES19DEL01,23.000000,4.400000,12.914264,77.678400,12.924264,77.688400,19-03-2022,8:30,8:45,Sandstorms,Low,0,Drinks,motorcycle,1.000000,No,Urban,26.000000
3,0x7a6a,COIMBRES13DEL02,38.000000,4.700000,11.003669,76.976494,11.053669,77.026494,05-04-2022,17:60,18:10,Sunny,Medium,0,Buffet,motorcycle,1.000000,No,Metropolitian,21.000000
4,0x70a2,CHENRES12DEL01,32.000000,4.600000,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30,13:45,Cloudy,High,1,Snack,scooter,1.000000,No,Metropolitian,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30.000000,4.800000,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,1,Meal,motorcycle,0.000000,No,Metropolitian,32.000000
45589,0xd641,AGRRES16DEL01,21.00,4.60,0.00,0.00,0.07,0.07,16-02-2022,19:55,20:10,Windy,Jam,0,Buffet,motorcycle,1.00,No,Metropolitian,36.00
45590,0x4f8d,CHENRES08DEL03,30.000000,4.900000,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50,24:05,Cloudy,Low,1,Drinks,scooter,0.000000,No,Metropolitian,16.000000
45591,0x5eee,COIMBRES11DEL01,20.000000,4.700000,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,0,Snack,motorcycle,1.000000,No,Metropolitian,26.000000


In [170]:
def Preprocessing(df):
    '''
    This function cleans data and performs some feature engineering technique
    on each of individual columns and prepares data for Machine Learning Model
    
    :param    df: Pandas DataFrame to be preprocessed
    
    :return   pd.DataFrame
    '''
     
    # Location co-ordinates in the given data cannot be negative value as well as zeroes
    # Replacing invalid data points in location co-ordinates i.e, o longitude and 0 latitude
    df[['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']]=\
                    df[['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude',
                                'Delivery_location_longitude']].replace(r'^0\W.*','NaN',regex=True)
    
    # Replacing 'NaN' strings to Numpy Null values
    df.replace('NaN',np.nan,inplace=True)
    
    # Imputing 'Time_orderd' Null values with 'Time_order_picked' values
    df['Time_Orderd'].fillna(df['Time_Order_picked'],inplace=True)
    
    # concating ['order_date' + 'time_ordered'] and ('order_date' + 'time_order_picked')
    df['Time_Orderd'] =df['Order_Date'] + " " + df['Time_Orderd']
    df['Time_Order_picked'] = df['Order_Date'] + " " + df['Time_Order_picked']
    
    #Converting to datetime dtype
    #Since some of the values need to be changed
        #'24:10' -> '00:10' [ParserError: hour must be in 0..23:]
        #'13:60' -> '14:00' [ParserError: minute must be in 0..59:]
    # Here, we simply adding ':00' seconds to time and converting it 'to_timedelta' will align according

    df['Time_Orderd']=\
        (pd.to_datetime(df['Time_Orderd'].str.split(" ").str[0] , dayfirst=True) + 
                                         pd.to_timedelta(df['Time_Orderd'].str.split(" ").str[1] + ":00"))
    df['Time_Order_picked'] = \
        (pd.to_datetime(df['Time_Order_picked'].str.split(" ").str[0], dayfirst=True) + 
                                        pd.to_timedelta(df['Time_Order_picked'].str.split(" ").str[1] + ":00"))
    
    # Imputing 'person_age' NULL Values with [Average] age within unique Person Id
    df['Delivery_person_Age']=df['Delivery_person_Age'].astype('float')
    grouped_mean_age= np.round(df.groupby('Delivery_person_ID')['Delivery_person_Age'].mean()).to_dict()
    df['Delivery_person_Age'].fillna(df['Delivery_person_ID'].map(grouped_mean_age),inplace=True)
    
    # Imputing 'Delivery_person_Ratings' NULL Values with [Average] age within unique Person Id
    df['Delivery_person_Ratings']=df['Delivery_person_Ratings'].astype('float')
    grouped_mean_ratings = np.round(df.groupby('Delivery_person_ID')['Delivery_person_Ratings'].mean(),1).to_dict()
    df['Delivery_person_Ratings'].fillna(df['Delivery_person_ID'].map(grouped_mean_ratings),inplace=True)
    
    # Sorting DataFrame, so that group of people within Time Frame will be used to impute null values
    df.sort_values(by=['Delivery_person_ID','Time_Orderd'],inplace=True)
    
    # Forward Filling Null values in each list of column
    fill_ = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude',
             'Weather conditions','Road_traffic_density','multiple_deliveries','Festival','City']
    
    for i in fill_:
        df[i].fillna(method='ffill',inplace=True)
    
    #Converting to numeric dtypes
    to_numeric =['Restaurant_latitude','Restaurant_longitude',
                 'Delivery_location_latitude','Delivery_location_longitude','multiple_deliveries',
                 'Vehicle_condition','Time_taken']

    for i in to_numeric:
        df[i] = df[i].astype('float')
    
    # Since location co-ordinates cannot be -ve, converting to absolute values
    to_abs = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']

    for i in to_abs:
        df[i]=df[i].abs()
    
    # Mapping Ordinal data column in Jam>High>Medium>Low
    traffic_map={'Low':1, 'Medium':2, 'High':3, 'Jam':4}
    df['Road_traffic_density']=df['Road_traffic_density'].map(traffic_map)
    
    # Elasped Time(min) between Time_ordered and Order_picked
    df['Order_Date']= (df['Time_Order_picked'] - df['Time_Orderd'])/np.timedelta64(1,'m')
    
    # One-Hot-encoding categorical variables
    df = pd.get_dummies(df,columns=['Weather conditions','Type_of_order','Festival'],drop_first=True)
    
    return df

In [171]:
train_df=Preprocessing(train_df)
train_df

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken,Weather conditions_Fog,Weather conditions_Sandstorms,Weather conditions_Stormy,Weather conditions_Sunny,Weather conditions_Windy,Type_of_order_Drinks,Type_of_order_Meal,Type_of_order_Snack,Festival_Yes
12437,0xdaa2,AGRRES010DEL01,34.0,4.5,27.163303,78.057044,27.173303,78.067044,10.0,2022-02-11 08:25:00,2022-02-11 08:35:00,1,0.0,motorcycle,0.0,Metropolitian,23.0,1,0,0,0,0,0,1,0,0
29279,0xd416,AGRRES010DEL01,36.0,4.9,27.163303,78.057044,27.203303,78.097044,5.0,2022-02-11 13:25:00,2022-02-11 13:30:00,3,0.0,motorcycle,1.0,Metropolitian,25.0,0,0,0,0,0,0,1,0,0
24595,0xe035,AGRRES010DEL01,20.0,4.8,27.163303,78.057044,27.233303,78.127044,5.0,2022-02-12 22:15:00,2022-02-12 22:20:00,1,1.0,motorcycle,0.0,Metropolitian,23.0,1,0,0,0,0,0,1,0,0
16556,0xd0d5,AGRRES010DEL01,31.0,4.8,27.163303,78.057044,27.173303,78.067044,0.0,2022-02-13 11:05:00,2022-02-13 11:05:00,1,2.0,scooter,0.0,Metropolitian,27.0,0,1,0,0,0,1,0,0,0
35450,0xc58f,AGRRES010DEL01,34.0,4.6,27.163303,78.057044,27.203303,78.097044,15.0,2022-02-13 16:30:00,2022-02-13 16:45:00,2,2.0,electric_scooter,1.0,Metropolitian,18.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12585,0x6fc3,VADRES20DEL03,24.0,4.8,22.311358,73.164798,22.401358,73.254798,15.0,2022-04-02 19:10:00,2022-04-02 19:25:00,4,2.0,motorcycle,1.0,Metropolitian,28.0,0,0,1,0,0,0,0,1,0
15943,0x463,VADRES20DEL03,32.0,4.7,22.311358,73.164798,22.371358,73.224798,0.0,2022-04-03 23:30:00,2022-04-03 23:30:00,4,3.0,electric_scooter,1.0,Metropolitian,21.0,0,0,1,0,0,0,0,0,0
29563,0x6c62,VADRES20DEL03,39.0,5.0,22.311358,73.164798,22.341358,73.194798,5.0,2022-04-05 19:00:00,2022-04-05 19:05:00,2,2.0,scooter,1.0,Urban,25.0,0,1,0,0,0,0,0,1,0
21995,0x4c10,VADRES20DEL03,37.0,4.9,22.311358,73.164798,22.371358,73.224798,15.0,2022-04-05 21:10:00,2022-04-05 21:25:00,4,1.0,motorcycle,1.0,Metropolitian,15.0,0,0,0,1,0,1,0,0,0


In [172]:
def create_target_and_predictors(df, target: str = "Time_taken"):
    """
    This function takes preprocessed data and splits the columns
    into a target column and a set of predictor variables, i.e. X & y.

    :param      df: dataframe containing data for the model
    :param      target: str, target variable that you want to predict

    :return     X: Numpy array
                y: Numpy array
    """
    
    df.drop(labels=['ID','Delivery_person_ID','Time_Orderd','Time_Order_picked','Type_of_vehicle','City'],axis=1,inplace=True)
    
    # Check to see if the target variable is present in the data
    if target not in df.columns:
        raise Exception(f"Target: {target} is not present in the data")
    
    X = df.drop(columns=[target],axis=1).values
    y = df[target].values
    
    return X, y

In [173]:
X,y = create_target_and_predictors(train_df,target='Time_taken')
print(X.shape, y.shape)

(45593, 19) (45593,)


In [174]:
def train_algorithm_with_cross_validation(X, y):
    """
    This function takes the predictor and target variables and
    trains a XGBoost Regressor model.

    :param      X: Numpy array, predictor variables
    :param      y: Numpy array, target variable

    :return
    """
    
    # Create training and test samples
    X_train, X_cross_val, y_train, y_cross_val = train_test_split(X, y, test_size=0.2)


    # Scale X data, we scale the data because it helps the algorithm to converge
    # and helps the algorithm to not be greedy with large values
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_cross_val = scaler.transform(X_cross_val)    
    
    # Instantiate algorithm 
    model = xgb.XGBRegressor(n_estimators=400,max_depth=10,learning_rate=0.2)

    # Train model
    model.fit(X_train, y_train, 
              eval_metric='rmse',
              early_stopping_rounds=10,
              eval_set=[(X_cross_val,y_cross_val)])

    # Generate predictions on Cross Validation sample
    y_pred = model.predict(X_cross_val)

    print(f'R^2 Cross Val:  {100*(r2_score(y_cross_val, y_pred))}')

In [175]:
def run():
    """
    This function executes the training pipeline of loading the prepared
    dataset from a text file and training the machine learning model

    :param

    :return
    """

    # Load the data first
    train_df = Data_Transforming('merged_file1.txt')
    
    train_df = Preprocessing(train_df)

    # Now split the data into predictors and target variables
    X, y = create_target_and_predictors(train_df,target='Time_taken')

    # Finally, train the machine learning model
    train_algorithm_with_cross_validation(X=X, y=y)

In [176]:
run()

[0]	validation_0-rmse:22.23069
[1]	validation_0-rmse:18.07864
[2]	validation_0-rmse:14.81647
[3]	validation_0-rmse:12.27047
[4]	validation_0-rmse:10.30171
[5]	validation_0-rmse:8.81764
[6]	validation_0-rmse:7.70230
[7]	validation_0-rmse:6.90241
[8]	validation_0-rmse:6.31723
[9]	validation_0-rmse:5.90985
[10]	validation_0-rmse:5.63251
[11]	validation_0-rmse:5.43532
[12]	validation_0-rmse:5.29876
[13]	validation_0-rmse:5.20449
[14]	validation_0-rmse:5.13671
[15]	validation_0-rmse:5.09573
[16]	validation_0-rmse:5.06681
[17]	validation_0-rmse:5.04246
[18]	validation_0-rmse:5.01736
[19]	validation_0-rmse:4.98978
[20]	validation_0-rmse:4.95345
[21]	validation_0-rmse:4.92699
[22]	validation_0-rmse:4.91530
[23]	validation_0-rmse:4.89781
[24]	validation_0-rmse:4.88377
[25]	validation_0-rmse:4.87771
[26]	validation_0-rmse:4.86216
[27]	validation_0-rmse:4.85673
[28]	validation_0-rmse:4.85182
[29]	validation_0-rmse:4.85381
[30]	validation_0-rmse:4.82942
[31]	validation_0-rmse:4.81686
[32]	validati

- Cross Validation Produces 75% Accuracy

### 

## Test Data Processing

In [97]:
import os
# find all the txt files in the dataset folder
inputs = []
for file in os.listdir("test"):
    if file.endswith(".txt"):
        inputs.append(os.path.join("test", file))
        
# concatanate all txt files in a file called merged_file.txt
with open('merged_test_file.txt', 'w') as outfile:
    for fname in inputs:
        with open(fname, encoding="utf-8", errors='ignore') as infile:
            for line in infile:
                outfile.write(line)
            
            outfile.write("\n")
            

In [148]:
# 'Data_Transforming' function has been altered. 
# Because there is no target variable in test data, the following code was deleted when calling this function.
# data['Time_taken']=data['a'].str.extract(r"^Time_taken\s\(.+\)\s+(.*)").dropna().reset_index(drop=True)

test_df=Data_Transforming('merged_test_file.txt')

In [150]:
# Preprocessing function has been altered 
# Time_taken column was erased, while calling the function
# to_numeric =[...................'Time_taken']

test_df=Preprocessing(test_df)

In [152]:
# Creating DataFrame with 'ID' column for submission file
ID_value=pd.DataFrame(test_df['ID'].tolist())

In [154]:
# Dropping all needless columns
test_df.drop(labels=['ID','Delivery_person_ID','Time_Orderd','Time_Order_picked','Type_of_vehicle','City'],axis=1,inplace=True)

In [155]:
# Scale test array for fast computing
X_test=scaler.fit_transform(test_df.values)

In [156]:
# Prediction on test values
prediction=np.round(model.predict(X_test))

In [157]:
# Creating DataFrame with 'Prediction' values for submission file
Prediction_value = pd.DataFrame(prediction)

### Submission

In [159]:
submission=pd.concat([ID_value,Prediction_value],
                     ignore_index=True,
                     axis=1)
submission.columns=['ID','Time_taken (min)']

In [177]:
submission.to_csv('test.csv', index=False)

### 

### T Rushti Rajoli
